In [ ]:
type Node{T}
    left::Nullable{Node{T}}
    right::Nullable{Node{T}}
    value::T
end

Node{T}(v::T) = Node(Nullable{Node{T}}(), Nullable{Node{T}}(), v)

In [1]:
n1 = Node(Nullable(Node(2)), Nullable(Node(3)), 10)

LoadError: LoadError: UndefVarError: Node not defined
while loading In[1], in expression starting on line 1

In [12]:
left(n::Node) = isnull(n.left) ? Void(): get(n.left)
right(n::Node) = isnull(n.right) ? Void(): get(n.right)

right (generic function with 1 method)

In [1]:
function search{T}(graph::Node{T}, number)
    for i = 1:10
        if graph.value == number
            return number
        else
            try
                graph = graph.left.value
            catch
                return 0
            end
        end
    end
end
    

LoadError: LoadError: UndefVarError: Node not defined
while loading In[1], in expression starting on line 1

In [1]:
include("SymmetricMatrix.jl")
include("Test.jl")
using Test

In [5]:
bs = (convert(BoxStructure{Float64}, st, 10)) ;
@time bs1 = (convert(BoxStructure{Float64}, st1, 10)) ;

  0.368854 seconds (320.15 k allocations: 589.424 MB, 32.72% gc time)


In [6]:
@time bs+bs1;
@time bs+2;
@time st + st1;
@time st + 2;

  0.107971 seconds (259.60 k allocations: 13.942 MB)
  0.080173 seconds (322.12 k allocations: 17.625 MB, 4.54% gc time)
  0.013778 seconds (7 allocations: 47.684 MB, 19.70% gc time)
  0.012109 seconds (7 allocations: 47.684 MB, 10.60% gc time)


In [7]:
@time convert(Array{Float64}, bs);
@time vec(bs);
@time vec(st);

  0.338190 seconds (1.23 M allocations: 148.155 MB, 15.07% gc time)
  0.297563 seconds (1.21 M allocations: 147.300 MB, 15.78% gc time)
  0.000001 seconds (6 allocations: 240 bytes)


In [8]:
@time bs.*bs1;
@time bs+bs1;
@time bs./bs1;
@time st.*st1;
@time st+st1;
@time st./st1;

  0.046923 seconds (217.14 k allocations: 12.077 MB, 5.30% gc time)
  0.041683 seconds (211.89 k allocations: 11.764 MB)
  0.046537 seconds (217.14 k allocations: 12.077 MB, 5.47% gc time)
  0.018239 seconds (13 allocations: 47.684 MB, 7.72% gc time)
  0.015898 seconds (7 allocations: 47.684 MB, 4.35% gc time)
  0.049982 seconds (13 allocations: 47.684 MB, 1.66% gc time)


In [8]:
@time modemult(bs, c', 4)
@time Tensors.modemult(st,c',4);

  3.900043 seconds (5.54 M allocations: 6.035 GB, 6.03% gc time)
  1.293228 seconds (176 allocations: 946.063 MB, 2.76% gc time)


In [1]:
using Tensors
using NullableArrays
using Iterators
using Base.size

include("SymmetricMatrix.jl")
using SymmetricMatrix
using NullableArrays
importall SymmetricMatrix

using Iterators
function genstensor(T::Type, dims::Int, l::Int, seed::Int = 1234)
    srand(seed)
    ret = zeros(T, fill(l, dims)...)
    indices = collect(product(fill(collect(1:l), dims)...))
    elements = Array{T,1}(randn(l^dims÷dims))
    r = 1
    for i in indices 
        if issorted(i)          
            for k in collect(permutations(i))
                ret[k...] = elements[r]
            end
            r += 1
        end
    end
    ret
end


genstensor (generic function with 2 methods)

In [ ]:
l = 75
st = genstensor(Float64, 4,l);
st1 = genstensor(Float64, 4,l, 1233);
c = randn(l,12);

In [2]:
T1 = randn(5,5);
r = randn(25, 25)
r1 = randn(25,10)
c = covbs(r, 5);
c1 = cov(r, corrected = false);



In [2]:
t = genstensor(Float64, 3,25)
bs = convert(BoxStructure{Float64}, t, 5);

In [29]:
seg(i::Int, of::Int, limit::Int) = (i*of <= limit)? (((i-1)*of+1):(i*of)): (((i-1)*of+1):limit)

@generated function slise1{T <: AbstractFloat, N}(mat::Array{T, N}, slisesize::Int)
    quote
        segments = ceil(Int, [size(mat)...]/slisesize)
        ret = NullableArray(Array{T, N}, segments...)
        @nloops $N i x -> 1:segments[x] begin
            ind = @ntuple $N x -> i_{$N-x+1}
            ret[ind...] = mat[(map(i -> seg(ind[i], slisesize, size(mat,i)), 1:N))...]
        end
        ret
    end
end


slise1 (generic function with 1 method)

In [112]:


mat = randn(6,6);
m = slise(mat, 3);
m1 = slise(mat', 3);
t = genstensor(Float64, 4,6)
bs = convert(BoxStructure{Float64}, t, 2);

In [113]:
function mult{T <: AbstractFloat, N}(bs::Union{BoxStructure{T, N}, 
        NullableArray{Array{T, N}, N}}, m::NullableArray{Matrix{T}}, j, n::Int, s::Int)
    temp = NullableArray(Array{T, N}, fill(s, N)...)
    if n > 1
        for i in product(fill(1:s, (n-1))...)
            temp[i...] = segmentmult1([i...,j...], bs, m, n)
        end
    elseif n == 1
        temp[1] = segmentmult1([j...], bs, m, n)
    end
    temp
end

mult (generic function with 1 method)

In [150]:
N = 4
ret = NullableArray(Array{Float64, N}, fill(s[2], N)...)
for i = 1:s[2], j = 1:i, k = 1:j, l = 1:k
    ret[l,k,j,i] = (mm(i,j,k)[l].value)
end
ret[2,1,1,1];

In [191]:
function mm(i::Int,j::Int,k::Int)
    s = size(bs)
    ret = bs
    ret = mult(bs, m, [i], 4, s[2]);
    ret = mult(ret, m, [j, 1], 3, s[2])
    mult(ret, m, [k, 1, 1], 2, s[2])
end

N = 4
ret = NullableArray(Array{Float64, N}, fill(s[2], N)...)
for i = 1:s[2], j = 1:s[2], k = 1:s[2], l = 1:s[2]
    ret[i,l,k,j] = (mm(i,j,k)[l].value)
end
ret[1,1,1,2];

2x2x2x2 NullableArrays.NullableArray{Array{Float64,4},4}:
[:, :, 1, 1] =
      3x3x3x3 Array{Float64,4}:
[:, :, 1, 1] =
 -22.7886   87.3779  -28.7993
  47.4691  -17.1373   97.6628
   3.2108  -25.607   -26.7401

[:, :, 2, 1] =
 107.095   -56.5826   -52.0874 
 -27.1129   97.8658    -6.22401
 -42.5507   -5.53224   33.2189 

[:, :, 3, 1] =
  -5.2751   -31.7583   25.6305
 -23.1081   -28.1404  -41.4878
   2.87276   20.8429    4.5511

[:, :, 1, 2] =
 107.095   -56.5826   -52.0874 
 -27.1129   97.8658    -6.22401
 -42.5507   -5.53224   33.2189 

[:, :, 2, 2] =
 -55.6779   72.2134    0.00141865
  35.3165  -32.7907  -31.6806    
  19.9935  -33.1008   -5.91456   

[:, :, 3, 2] =
 -44.9248     0.779416   39.219  
  -5.12518  -34.2247      2.25109
  24.3152     1.4902    -10.0498 

[:, :, 1, 3] =
  -5.2751   -31.7583   25.6305
 -23.1081   -28.1404  -41.4878
   2.87276   20.8429    4.5511

[:, :, 2, 3] =
 -44.9248     0.779416   39.219  
  -5.12518  -34.2247      2.25109
  24.3152     1.4902    -10.

In [189]:
r = t
for i = 2:4
    r = Tensors.modemult(r, mat, i)
end

slise1(r, 3)[2,1,1,1]

Nullable(3x3x3x3 Array{Float64,4}:
[:, :, 1, 1] =
 -43.1523  -18.0105    27.1124
 -18.0105   -1.89069   12.0542
  27.1124   12.0542   -18.0902

[:, :, 2, 1] =
 -18.0105    -1.89069  12.0542 
  -1.89069  -16.5722   11.0968 
  12.0542    11.0968   -7.57523

[:, :, 3, 1] =
  27.1124  12.0542   -18.0902 
  12.0542  11.0968    -7.57523
 -18.0902  -7.57523   10.9983 

[:, :, 1, 2] =
   1.41346  -95.6701  14.6193 
 -95.6701    62.2032  37.1311 
  14.6193    37.1311  -3.07002

[:, :, 2, 2] =
 -95.6701   62.2032   37.1311
  62.2032  -52.7219  -57.7438
  37.1311  -57.7438   15.1439

[:, :, 3, 2] =
 14.6193    37.1311   -3.07002
 37.1311   -57.7438   15.1439 
 -3.07002   15.1439  -34.0501 

[:, :, 1, 3] =
 -124.727    22.1264   52.0471 
   22.1264   51.0128  -43.7799 
   52.0471  -43.7799    2.46936

[:, :, 2, 3] =
  22.1264    51.0128  -43.7799
  51.0128  -102.992    26.7913
 -43.7799    26.7913   27.5347

[:, :, 3, 3] =
  52.0471   -43.7799    2.46936
 -43.7799    26.7913   27.5347 
   2.46936 

In [6]:
a*0

SymmetricMatrix.BoxStructure{Float64,2}(NullableArrays.NullableArray{Array{Float64,2},2}[
3x3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0,

#NULL,#NULL,
3x3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0,

3x3 Array{Float64,2}:
  0.0  -0.0  -0.0
 -0.0   0.0  -0.0
 -0.0  -0.0   0.0,

#NULL,
3x1 Array{Float64,2}:
 0.0
 0.0
 0.0,

3x1 Array{Float64,2}:
 -0.0
 -0.0
  0.0,

1x1 Array{Float64,2}:
 0.0],3)

In [8]:
*(a, 0, true)


SymmetricMatrix.BoxStructure{Float64,2}(NullableArrays.NullableArray{Array{Float64,2},2}[
3x3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0,

#NULL,#NULL,
3x3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0,

3x3 Array{Float64,2}:
  0.0  -0.0  -0.0
 -0.0   0.0  -0.0
 -0.0  -0.0   0.0,

#NULL,
3x1 Array{Float64,2}:
 0.0
 0.0
 0.0,

3x1 Array{Float64,2}:
 -0.0
 -0.0
  0.0,

1x1 Array{Float64,2}:
 0.0],3)

In [9]:
a

SymmetricMatrix.BoxStructure{Float64,2}(NullableArrays.NullableArray{Array{Float64,2},2}[
3x3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0,

#NULL,#NULL,
3x3 Array{Float64,2}:
  0.0  -0.0   0.0
 -0.0   0.0  -0.0
  0.0  -0.0   0.0,

3x3 Array{Float64,2}:
  0.0  -0.0  -0.0
 -0.0   0.0  -0.0
 -0.0  -0.0   0.0,

#NULL,
3x1 Array{Float64,2}:
 0.0
 0.0
 0.0,

3x1 Array{Float64,2}:
 -0.0
 -0.0
  0.0,

1x1 Array{Float64,2}:
 0.0],3)

In [7]:
j = 2
k = [1,2,3]
[j, k[2:end]...]

3-element Array{Int64,1}:
 2
 2
 3

In [77]:
[1,2,3][end:-1:1]

3-element Array{Int64,1}:
 3
 2
 1